Ch5. 컴퓨터 비전을 위한 딥러닝
----------

### 5.1 Introduction to convnets

convnets(컨브넷) 이라고 불리는 convolution neural network(합성곱 신경망)에 대한 학습

우선 예제를 살펴보자.

In [1]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from keras import models, layers

%matplotlib inline

Using TensorFlow backend.


In [5]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [6]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


**convnets**이 (image_height, image_width, image_channels) 크기의 입력 텐서를 사용한다는 점이 중요
- 예제에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하도록 컨브넷을 설정해야 함
- 그래서 첫 layer의 parameter input_shape=(28, 28, 1)를 설정함

Conv2D와 MaxPooling2D층의 출력은 (height, width, channels) 크기의 3D 텐서이다.
- 높이와 넓이 차원은 네트워크가 깊어질수록 작아지는 경향이 있다.
- 채널의 수는 Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절됨(32개 또는 64개)

다음 단계에서 마지막 층의 ((3, 3, 64) 크기) 추력 텐서를 fully connected network에 넣는다.
- 분류기는 1D 벡터를 처리하는데 이전 출력이 3D 텐서
- 그래서 3D 출력을 1D 텐서로 펼쳐야함
- 그 다음 몇 개의 Dense 층을 추가함

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [8]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

(3, 3, 64) -> (576,) 벡터로 펼쳐진 후 Dense로 넘어감

In [2]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [17]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [18]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 20s 337us/step - loss: 0.1775 - accuracy: 0.9438
Epoch 2/5
60000/60000 [==============================] - 19s 312us/step - loss: 0.0473 - accuracy: 0.9852
Epoch 3/5
60000/60000 [==============================] - 19s 313us/step - loss: 0.0330 - accuracy: 0.9898
Epoch 4/5
60000/60000 [==============================] - 19s 321us/step - loss: 0.0242 - accuracy: 0.9929
Epoch 5/5
60000/60000 [==============================] - 19s 314us/step - loss: 0.0193 - accuracy: 0.9939


In [19]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 1s 128us/step


0.9905999898910522

**왜 컨브넷이 더 좋은 퍼포먼스를....?**

#### 5.1.1 합성곱 연산

근본적인 차이
- Dense layer는 입력 특성 공간에 있는 전역 패턴(예를 들면 MNIST 숫자 이미지에서는 모든 픽셀에 걸친 패턴)을 학습하지만 합성곱 층은 지역 패턴을 학습
- 이미지일 경우 작은 2D window로 입력에서 패턴을 찾음
- 앞의 예에서는 3x3 크기였음

핵심적인 특정인 두 가지 성질을 알려줌
- **학습된 패턴은 평행 이동 불변성(translation invariant)를 가짐**
  - 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳에서도 이 패턴을 인식 할 수 있다.
  - 완전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야함(상대적으로 안 좋은 이유임)
  - 이런 성질 때문에 컨브넷이 이미지를 효율적으로 처리하게 만들어 줌(근본적으로 우리가 보는 세상은 평행 이동으로 인해 다르게 인식 되지 않음..결국 사람이네)
  - 적은 수의 훈련 샘플을 사용해서 일반화 능력을 가진 표현을 학습할 수 있다.


- **컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있다**
  - 첫 번째 합성곱층이 에지 같은 작은 지역 패턴을 학습함
  - 두 번째 합성곱층이 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습하는 식
  - 이런 방식을 사용하여 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있다.
    - 근본적으로 우리가 보는 세상은 공간적 계층 구조를 가짐
    

![alt text](https://www.researchgate.net/profile/Hassan_Hassan45/publication/337830197/figure/fig3/AS:834037371379712@1575861413309/Schematic-diagram-of-the-proposed-edge-detectors-using-simple-CNN-setting.jpg)

합성곱 연산은 feature map(특성 맵)이라고 부르는 3D 텐서에 적용
- 이 텐서는 2개의 공간 축(높이와 넓이)과 깊이 축(채널 축)으로 구성 됨
- RGB이미지는 3개의 컬러 채널(빨, 녹, 파)을 가지므로 깊이 축의 차원이 3이 됨
- MNIST 숫자처럼 흑백 이미지는 깊이 축의 차원이 1(회색 톤)이다.
- 합성곱 연산은 입력 특성 맵에서 작은 patch를 추출하고 이런 모든 패치에 같은 변환을 적용하여 output feature map(출력 특성 맵)을 만듬

**output feature map**도 높이와 넓이를 가진 3D 텐서
- 출력 텐서의 깊이는 층의 매개변수로 결정되므로 상황에 따라 다름
- 이렇게 되면 깊이 축의 채널은 더 이상 RGB 입력처럼 특정 컬러를 의미하지 않음
- 그 대신 일종의 filter(필터)를 의미
- MNIST 예제에서는 첫 번째 합성곱 층이 (28, 28, 1) 크기의 특성 맵을 입력으로 받아 (26, 26, 32)크기의 특성 맵을 출력, 이 값은 입력데 대한 필터의 response map(응답 맵) 이다. 입력의 각 위체엇 필터 패턴에 대한 응답을 나타냄
  - 즉 입력에 대해 32개의 필터를 적용
  - 필터 하나의 크기는 (patch_height, patch_width, input_depth)
  - 첫 번째 합성곱은 (3, 3, 1) 크기의 필터를 32개 적용하고, 두 번쨰 합성곱은 (3, 3, 32) 크기의 필터를 64개 적용
- 특성 맵이란 말이 의미하는 것은....
  - 깊이 축에 있는 각 차원은 하나의 특성(또는 필터)이고, 2D 텐서 output[:, :, n]은 입력에 대한 이 필터 응답을 나타내는 2D 공간상의 맵
  

![alt text](https://i.ytimg.com/vi/HMcx-zY8JSg/maxresdefault.jpg)

합성곱은 핵심적인 2개의 parameter로 정의
- 입력으로부터 뽑아낼 패치의 크기 : 전형적으로 3X3 또는 5X5 크기를 사용, 이 예에서는 일반적으로 많이 사용하는 3X3 크기를 사용
- 특성 맵의 출력 깊이 : 합성곱으로 계산할 필터의 수, 이 예에서는 깊이 32로 시작해서 깊이 64로 끝남


- 3D 입력 특성 맵 위를 3 X 3 또는 5 X 5 window가 슬라이딩 하면서 모든 위치에서 3D 특성 패치((window_height, window_width, input_depth))를 추출하는 방식으로 합성곱이 작동
- 위의 결과는 1D 벡터로 변환
- 변환된 모든 벡터는 (height, width, output_depth) 크기의 3D 특성 맵으로 재구성
  - 출력 높이, 넓이는 입력 높이, 넓이와 다를 수 있다.
    - 경계 문제, 입력 특성 맵의 패딩을 추가하여 대응 할 수 있기 때문이다.
    - stride 의 사용 여부에 따라 다르다.
    
    
좋은 참고 자료
- https://je-d.tistory.com/entry/%ED%95%A9%EC%84%B1%EA%B3%B1-%EC%8B%A0%EA%B2%BD%EB%A7%9DCNN
- http://taewan.kim/post/cnn/

**경계 문제와 패딩 이해하기**

- 입력과 동일한 높이와 넓이를 가진 feature map을 얻고 싶다면 padding 을 사용할 수 있음
  - paramter는 2개의 값이 가능
  - valid 는 패딩을 사용하지 않는다는 뜻
  - same 은 입력과 동일한 높이와 넓이를 가진 출력을 만들기 위해 패딩한다 라는 뜻
  - 기본 매개변수는 valid 이다.

![alt text](https://miro.medium.com/max/666/1*noYcUAa_P8nRilg3Lt_nuA.png)

**합성곱 스트라이드 이해하기**

출력 크기에 영향을 미치는 다른 요소는 스트라이드
- 위에서의 설명들은 윈도우가 중앙 타일이 연속적으로 지나간다고 가정한 것
- 두 번의 연속적인 윈도우 사이의 거리가 스트라이드라고 불리는 합성곱의 파라미터이다.
- 기본 값은 1
- 1보다 큰 스트라이드 합성곱도 가능